Azure > Suscription > Resource group > Workspace

## WORKSPACES
Los espacios de trabajo son contextos para las tareas de aprendizaje automático.

Contienen activos relacionados, como experimentos, datos y objetivos de cómputo.

Componentes de un Espacio de Trabajo:
- **Objetivos de Cómputo** = Destinos para entrenamiento y despliegue.
- **Datos** = Utilizados para experimentación y entrenamiento de modelos.
- **Notebooks** = Contienen código compartido y documentación.
- **Experimentos** = Historial de ejecución con métricas y resultados registrados.
- **Tuberías (Pipelines)** = Definen procesos orquestados de varios pasos.
- **Modelos Entrenados**

Recursos de Azure Asociados al Workspace:
- **Cuenta de Almacenamiento** = Almacena archivos y datos para experimentos y entrenamiento.
- **Application Insights** = Monitorea servicios predictivos en el espacio de trabajo.
- **Azure Key Vault** = Gestiona secretos como claves de autenticación.
- **Registro de Contenedores** = Creado según sea necesario para administrar modelos desplegados.
- **Políticas de Autorización Basadas en Roles** = Restringe acciones específicas para usuarios y grupos.


In [ ]:
# conexion al Workspace mediante SDK

from azure.core Import Workspace
ws = Workspace.create(
    name = 'myworkspace',
    subscription_id = 'subscription-id',
    resource_group = 'myresourcegroup',
    create_resource_group = True,
    location = 'eastus'
)


# Conexion al Workspace mediante CLI
az ml workspace create -w 'myworkspace' -g 'myresourcegroup'    #-w (workspace) -g (resource group)

### Administrar Workspaces

#### 1. Interfaces
- **Azure Portal**: Interfaz general para todos los recursos de Azure, puede ser poco intuitiva para el aprendizaje automático, interesante para DevOps.
- **Azure Machine Learning Studio**: Interfaz web dedicada a la gestión de workspaces de Azure Machine Learning, de facil acceso.
    - Herramientas:
      - Designer: Interfaz de arrastrar y soltar para el desarrollo de modelos sin código.
      - Aprendizaje Automático Automatizado: Asistente para entrenar modelos con diferentes algoritmos y preprocesamiento de datos.
    -  Programación:
       - Ventaja: Automatización, repetibilidad, consistencia y CICD.

- **SDKs**: Python y R (nosotros Python).
  - Instalación con pip.
  - Conexión al workspace:
    - Archivo de configuración.
      1. Método "from_config" de la clase Workspace.
      2. Método "Get" de la clase Workspace (menos flexible).

- **Azure CLI**: Herramienta de línea de comandos multiplataforma.
  - Requiere extensión de Azure Machine Learning CLI.
  - Comandos "az ml" con parámetros.

- **Entornos de Desarrollo**:
  1. **Compute Targets**: Instancias informáticas dentro del workspace.
  2. **Jupyter Notebook/Lab**: Instalaciones preconfiguradas en compute targets para usar el SDK.
  3. **Visual Studio Code**: Editor de código con extensiones para Python y Azure Machine Learning.

#### 2. Experimentos en AzureML

Concepto:
- proceso con nombre que ejecuta un script o pipeline.
- Genera métricas y salidas guardadas en el workspace.
- Permite múltiples ejecuciones con diferentes datos, código o configuraciones.
- AzureML logea cada ejecución.

Control de experimentos:
- Mediante el contexto de ejecución del experimento.
- Visualización de detalles en la pestaña de experimentos de AzureML Studio.

Registro de métricas:
- Archivos de registro con mensajes desde la terminal.
- Funciones del objeto run para registrar métricas con nombre:
    - `log`: Valor único con nombre.
    - `log_list`: Lista con nombre.
    - `log_row`: Fila con múltiples columnas.
    - `log_table`: Diccionario como tabla.
    - `log_image`: Imagen o gráfico.
- Visual en Azure ML Studio o mediante el widget RunDetails.
- Recuperación mediante el método get_metrics del objeto run.

In [ ]:
## Codigo para registrar el numero de entrada en un csv en un experimento

from azureml.core import Experiment
import pandas as pd

# Crear un experimento en el workspace
experiment = Experiment(workspace = ws, name= "myexperiment")

# Iniciar el registro de datos en el experimento
run = experiment.start_logging()

# Cargar los datos y contar las filas
data = pd.read_csv('data.csv')
row_count = (len(data))

# Registrar el número de filas
run.log('observations', row_count)
run.complete()

In [ ]:
# Recuperar metricas con Widget RunDetails.

from azureml.widgets import RunDetails
RunDetails(run).show()

In [ ]:

# Recuperar metricas con el metodo run object get_metrics
import json


metrics = run.get_metrics()
print(json.dumps(metrics, indent=2))

Generación de archivos de salida:

modelos entrenados o cualquier tipo de archivo.
- Guardados en la carpeta outputs del experimento.
- Técnicas de carga de archivos según la ejecución del experimento:
    - Ejecución en línea: upload_file del objeto run.
    - Ejecución remota: Automática al finalizar la ejecución.
- Recuperación mediante el método get_outputs del objeto run.

In [ ]:
# subir archivo con el objeto run
run.upload_file(name='outputs/sample.csv', path_or_stream='./sample.csv')

In [ ]:
# Recuperar archivos con el metodo run object get_file_names
files = run.get_file_names()
print(json.dumps(files, indent=2))

Ejecución de experimentos:
1. En línea mediante start_logging del objeto experiment.
2. En script para mayor flexibilidad y escalabilidad.

In [ ]:
from azureml.core import Run
import pandas as pd
import matplotlib.pyplot as plt
import os

run = Run.get_context()

data = pd.read_csv('data.csv')

row_count = (len(data))
run.log('observations', row_count)

# Crear un directorio para los archivos de salida
os.makedirs('outputs', exist_ok=True)
data.sample(100).to_csv("outputs/sample.csv", index=False, header=True)

run.complete()

Scripts de experimento:
- Archivos Python con el código a ejecutar.
- Importan azureml.core.run y utilizan get_context para acceder al contexto.
- Utilizan el contexto para registrar métricas, subir archivos y finalizar el experimento.

Configuración correcta de los scripts:
- Definida por ScriptRunConfig: script a ejecutar y entorno Python.
- Ejemplo: ejecutar script en carpeta específica con archivos dependientes (ej. data.csv).
- Entorno Python definido por:
    - Entorno predeterminado con paquetes disponibles.
    - Entorno asociado a un objeto CondaDependencies para especificar paquetes adicionales.

In [ ]:
from azureml.core import Experiment, ScriptRunConfig

# Crear un objeto de configuracion de ejecucion de script
script_config = ScriptRunConfig(source_directory='experiment_folder', script='experiment.py')

# Crear un experimento
experiment = Experiment(workspace = ws, name = 'experiment_name')
run = experiment.submit(config=script_config)
run.wait_for_completion(show_output=True)